<a href="https://colab.research.google.com/github/BruceAuHK/Dashboard_FFO/blob/master/Plotly_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import urllib.request
import json
import ssl
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from scipy import special 
import pandas as pd
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

context = ssl._create_unverified_context()

def get_data_from_HKMA(url):
    # Balance sheet
    url = url

    # retrieve data from hkma api, 100 entries per call, untill every data are retrieveed
    records = []
    offset = 0
    while True:
        with urllib.request.urlopen(url+"&offset="+str(offset), context=context) as req:
            data = json.loads(req.read().decode())
            records += data['result']['records']
        if data['result']['datasize'] <= 0:
            break
        else:
            offset += 100

    # now data retrieved from hkma are stored in 'records' in a chronological order
    return records


print("dependencies ready")


dependencies ready


# **Monetary Base**

A part of the monetary liabilities of a central bank. The Monetary Base is defined, at the minimum, as the sum of the currency in circulation (banknotes and coins) and the balance of the banking system held with the central bank (the reserve balance or the clearing balance). In Hong Kong, the Monetary Base comprises Certificates of Indebtedness (for backing the banknotes issued by the note-issuing banks), government-issued currency in circulation, the balance of the clearing accounts of banks kept with the HKMA (the Aggregate Balance), and Exchange Fund Bills and Notes.


In [112]:
MBC = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/monetary-operation/monetary-base-endperiod?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#1. Size of the Monetary Base was published since 25 November 1998.
mbc_df = pd.json_normalize(get_data_from_HKMA(MBC))

#Convert dataframe to dataframe long format
mbc_dfl = pd.melt(mbc_df, id_vars = ['end_of_month'], value_vars =['cert_of_indebt', 'gov_notes_coins_circulation' , 'aggr_balance_bf_disc_win' , 'outstanding_efbn' , 'ow_lb_bf_disc_win' , 'mb_bf_disc_win_total'],var_name='Size of the Monetary Base', value_name='HK$ million' )

fig = px.area(mbc_dfl, x='end_of_month' , y='HK$ million', color = 'Size of the Monetary Base')
fig.update_layout(title_text="Monetary Base – End of period figures")
fig.show()


# 1. Money Multiplier = M3 / MB (HKD)

The total stock of money available in the economy. Hong Kong has three measures of money supply:

Money Supply definition 1 (M1): The sum of legal tender notes and coins held by the public plus customers' demand deposits placed with banks.

Money Supply definition 2 (M2): M1 plus customers' savings and time deposits with banks plus negotiable certificates of deposit (NCDs) issued by banks held outside the banking sector.

Money Supply definition 3 (M3): M2 plus customers' deposits with restricted licence banks and deposit-taking companies plus NCDs issued by these institutions held outside the banking sector.

Among these three series, HK M1 exhibits a significant seasonal pattern, whereas there is no strong evidence of seasonality in broad money (HK M2 and HK M3). Seasonally adjusted series of HK M1 and its components (i.e. cash held by the public and demand deposits) are compiled and published by the HKMA.

In [113]:
# ---------------- 1. Money Multiplier = M3 / MB (HKD) ----------------

MSA = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/money/supply-adjusted?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#1. There is a break in data series due to the inclusion of short-term Exchange Fund placements of less than one month since April 1997.
msa_df = pd.json_normalize(get_data_from_HKMA(MSA))


In [114]:
m3_hkd = msa_df[['end_of_month','m3_total']]

mb_tot = mbc_df[['end_of_month','mb_bf_disc_win_total']]

m3_hkd_Over_mb_tot = pd.merge(m3_hkd, mb_tot, on='end_of_month')

m3_hkd_Over_mb_tot['m3Overmb'] = (m3_hkd_Over_mb_tot['m3_total'] / m3_hkd_Over_mb_tot['mb_bf_disc_win_total'])


fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "m3Overmb",
              
              labels={
                     "x": "End_of_month",
                     "y": "m3Overmb"
                 },
              title='M3(hkd) / MB(Total)')
fig.show()



In [115]:
fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "m3_total",
              
              labels={
                     "x": "End_of_month",
                     "y": "m3_total"
                 },
              title='M3(Total)')
fig.show()

In [116]:
fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "mb_bf_disc_win_total",
              
              labels={
                     "x": "End_of_month",
                     "y": "mb_bf_disc_win_total"
                 },
              title='Total Monetary Base before Discount Window')
fig.show()

# 2. Total deposits & M3 (HKD)

In [117]:
TD = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/customer-deposits-by-currency?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

td_records_df = pd.json_normalize(get_data_from_HKMA(TD))

deposits_all = td_records_df[['end_of_month','deposits_all']]

m3_hkd_and_deposits_all  = pd.merge(m3_hkd, deposits_all, on='end_of_month')
#Convert dataframe to dataframe long format
m3_hkd_and_deposits_all_dfl = pd.melt(m3_hkd_and_deposits_all, id_vars = ['end_of_month'], value_vars =['m3_total', 'deposits_all'],var_name='var', value_name='HK$ million' )

fig = px.area(m3_hkd_and_deposits_all_dfl, x='end_of_month' , y='HK$ million', color = 'var')

fig.update_layout(title_text="Total deposits & M3 (HKD)")

fig.show()

# 3. Loan to Deposits


In [118]:
BS = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/balance-sheet-ais?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

bs_records_df = pd.json_normalize(get_data_from_HKMA(BS))


In [119]:
bs_records_df['LtoD_hkd'] = (bs_records_df['asst_loanadvtocust_hkd'] / bs_records_df['liab_depcust_hkd'])

fig = px.line(bs_records_df, x="end_of_month", y = "LtoD_hkd",
              title='Loan to Deposits(hkd only)')
fig.show()

In [120]:
#Assets Loans and advances to customers HK$ - asst_loanadvtocust_hkd
#	Liabilities Deposits from customers HK$ -liab_depcust_hkd

#Convert dataframe to dataframe long format
bs_records_dfl = pd.melt(bs_records_df, id_vars = ['end_of_month'], value_vars =['asst_loanadvtocust_hkd', 'liab_depcust_hkd'],var_name='var', value_name='HK$ million' )

fig = px.line(bs_records_dfl, x='end_of_month' , y='HK$ million', color = 'var')

fig.update_layout(title_text="Loan(hkd) and  Deposits (hkd)")

fig.show()

## 3.2. Loan to Deposits (FC)


In [121]:

bs_records_df['LtoD_fc'] = (bs_records_df['asst_loanadvtocust_fc'] / bs_records_df['liab_depcust_fc'])

fig = px.line(bs_records_df, x="end_of_month", y = "LtoD_fc",
              title='Loan to Deposits(fc only)')
fig.show()

In [122]:
#Convert dataframe to dataframe long format
bs_records_dfl = pd.melt(bs_records_df, id_vars = ['end_of_month'], value_vars =['asst_loanadvtocust_fc', 'liab_depcust_fc'],var_name='var', value_name='HK$ million' )

fig = px.line(bs_records_dfl, x='end_of_month' , y='HK$ million', color = 'var')

fig.update_layout(title_text="Loan(fc) and  Deposits (fc)")

fig.show()

## 3.3 Loan to Deposits (Total)

In [123]:
bs_records_df['LtoD_total'] = (bs_records_df['asst_loanadvtocust_total'] / bs_records_df['liab_depcust_total'])

fig = px.line(bs_records_df, x="end_of_month", y = "LtoD_total",
              title='Loan to Deposits (Total)')
fig.show()

In [124]:
#Convert dataframe to dataframe long format
bs_records_dfl = pd.melt(bs_records_df, id_vars = ['end_of_month'], value_vars =['asst_loanadvtocust_total', 'liab_depcust_total'],var_name='var', value_name='HK$ million' )

fig = px.line(bs_records_dfl, x='end_of_month' , y='HK$ million', color = 'var')

fig.update_layout(title_text="Loan(Total) and  Deposits (Total)")

fig.show()


#4.Centaline index

Announced every month, latest on 2020/ 07/ 18, reflecting secondary private residential property price for 2020/05 (based on formal sale & purchase date; on average, formal S&P are signed within 14 days after preliminary S&P)

In [125]:
!wget https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/cci.csv

--2020-07-20 08:11:23--  https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/cci.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4205 (4.1K) [text/plain]
Saving to: ‘cci.csv.2’

cci.csv.2           100%[===================>]   4.11K  --.-KB/s    in 0s      

2020-07-20 08:11:23 (51.1 MB/s) - ‘cci.csv.2’ saved [4205/4205]



In [126]:
cci = pd.read_csv('cci.csv')

fig = px.line(cci, x="date", y = "cci",
              title='Centaline index')
fig.show()

#5.MB / AB vs HIBOR

## MB VS HIBOR

In [127]:
HIBOR = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/hk-interbank-ir-endperiod?sortby=end_of_month&sortorder=asc'
HIBOR_df = pd.json_normalize(get_data_from_HKMA(HIBOR))

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=HIBOR_df["end_of_month"], y = HIBOR_df["ir_1m"], mode='lines', name="HIBOR"),secondary_y=False)   #percent per annum

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['mb_bf_disc_win_total'], mode='lines', name="MB"),secondary_y=True,)   #HK$ million

# Add figure title
fig.update_layout( xaxis_range=['1998-7','2020-06'],
    title_text="MB VS HIBOR"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>percent per annum</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)

fig.show()

 


## AB VS HIBOR

### Aggregate Balance

The sum of balances in the clearing accounts and reserve accounts maintained by commercial banks with the central bank. In Hong Kong, this refers to the sum of the balances in the clearing accounts maintained by the banks with the HKMA for settling interbank payments and payments between banks and the HKMA. The Aggregate Balance is a part of the Monetary Base.

In [128]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=HIBOR_df["end_of_month"], y = HIBOR_df["ir_1m"], mode='lines', name="HIBOR"),secondary_y=False)   #percent per annum

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['aggr_balance_bf_disc_win'], mode='lines', name="AB"),secondary_y=True,)   #HK$ million

# Add figure title
fig.update_layout( xaxis_range=['1998-7','2020-06'],
    title_text="AB VS HIBOR"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>percent per annum</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)

fig.show()

 


#6. FPS

## Transactions and Registrations
### (Developping)

In [129]:
cht = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-ch-turnover?sortby=end_of_month&sortorder=asc'
cht_df = pd.json_normalize(get_data_from_HKMA(cht))

cht_df['tot'] = cht_df['total_hkd'] + cht_df['total_usd'] + cht_df['total_euro'] + cht_df['total_rmb'] 

fig = go.Figure()
fig.add_trace(go.Bar(x=cht_df['end_of_month'], y = cht_df['tot']))


# Use date string to set xaxis range
fig.update_layout(xaxis_range=['2018-8','2020-06'],
                  title_text="Clearing House Turnover")
fig.show()




## FPS Addressing Service Registrations Total Grand-total

In [130]:
fps_reg = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-fps-reg?sortby=end_of_month&sortorder=asc'
fps_reg_df = pd.json_normalize(get_data_from_HKMA(fps_reg))

fig = go.Figure()
fig.add_trace(go.Scatter(x=fps_reg_df['end_of_month'], y = fps_reg_df['total_grand_total']))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=['2018-8','2020-06'],
                  title_text="FPS Addressing Service Registrations Total Grand-total")
fig.show()




## FPS Turnover of HKD FPS payment (Value)

In [131]:
fps_val_hkd = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-turnover-fps-hkd-payment-val?sortby=end_of_month&sortorder=asc'
fps_val_hkd_df = pd.json_normalize(get_data_from_HKMA(fps_val_hkd))

fig = go.Figure()
fig.add_trace(go.Bar(x=fps_val_hkd_df['end_of_month'], y = fps_val_hkd_df['total']))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=['2018-9','2020-06'],
                  title_text="FPS Turnover of HKD FPS payment (Value in HK$ thousand)")
fig.show()


## FPS Turnover of RMB FPS payment (Value)

In [132]:
fps_val_rmb = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-turnover-fps-rmb-payment-val?sortby=end_of_month&sortorder=asc'
fps_val_rmb_df = pd.json_normalize(get_data_from_HKMA(fps_val_rmb))

fig = go.Figure()
fig.add_trace(go.Bar(x=fps_val_rmb_df['end_of_month'], y = fps_val_rmb_df['total']))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=['2018-9','2020-06'],
                  title_text="FPS Turnover of RMB FPS payment (Value in RMB thousand)")
fig.show()


#For Development

## Clearing House Turnover (3.12.1)	

In [133]:
cht = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-ch-turnover?sortby=end_of_month&sortorder=asc'
cht_df = pd.json_normalize(get_data_from_HKMA(cht))

cht_df['chq_tot'] = cht_df['chq_hkd'] + cht_df['chq_usd'] + cht_df['chq_rmb']
cht_df['chats_tot'] = cht_df['chats_hkd'] + cht_df['chats_usd'] + cht_df['chats_euro'] + cht_df['chats_rmb']
cht_df['ecg_tot'] = cht_df['ecg_hkd'] + cht_df['ecg_usd'] + cht_df['ecg_rmb']
cht_df['tot'] = cht_df['total_hkd'] + cht_df['total_usd'] + cht_df['total_euro'] + cht_df['total_rmb'] 

#Convert dataframe to dataframe long format
cht_dfl = pd.melt(cht_df, id_vars = ['end_of_month'], value_vars =['chq_tot', 'chats_tot', 'ecg_tot', 'tot'],var_name='var', value_name='HK$ million' )

fig = px.area(cht_dfl, x='end_of_month' , y='HK$ million', color = 'var')

# Use date string to set xaxis range
fig.update_layout(xaxis_range=['2007-05','2020-06'],
                  title_text="3.12.1	Clearing House Turnover")

fig.show()

##Faster Payment System Addressing Service Registrations(3.12.2)

In [134]:
fps_reg = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-fps-reg?sortby=end_of_month&sortorder=asc'
fps_reg_df = pd.json_normalize(get_data_from_HKMA(fps_reg))

#Convert dataframe to dataframe long format
fps_reg_dfl = pd.melt(fps_reg_df, id_vars = ['end_of_month'], value_vars =['mobile_personal', 'mobile_corporate',
       'mobile_subtotal', 'email_personal', 'email_corporate',
       'email_subtotal', 'fpsid_personal', 'fpsid_corporate', 'fpsid_subtotal',
       'total_personal', 'total_corporate', 'total_grand_total' ],var_name='var', value_name='Count' )

fig = px.area(fps_reg_dfl, x='end_of_month' , y='Count', color = 'var')

fig.update_layout(title_text="Faster Payment System Addressing Service Registrations(3.12.2)")

fig.show()

##Turnover of HKD FPS payment (Volume)(3.12.3)

In [ ]:
fps_vol = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-turnover-fps-hkd-payment-vol?sortby=end_of_month&sortorder=asc'
fps_vol_df = pd.json_normalize(get_data_from_HKMA(fps_vol))

#Convert dataframe to dataframe long format
fps_vol_dfl = pd.melt(fps_vol_df, id_vars = ['end_of_month'], value_vars =['rtctp_pp_payee_proxy_id', 'rtctp_pp_payee_acc_no',
       'rtctp_other_payment', 'rtctp', 'rtddp', 'rt_payment', 'bat_payment',
       'total'],var_name='var', value_name='Volume' )

fig = px.area(fps_vol_dfl, x='end_of_month' , y='Volume', color = 'var')

fig.update_layout(title_text="Turnover of HKD FPS payment (Volume)(3.12.3)")

fig.show()



##Turnover of HKD FPS payment (Value)(3.12.4	)

In [ ]:
fps_val = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-turnover-fps-hkd-payment-val?sortby=end_of_month&sortorder=asc'
fps_val_df = pd.json_normalize(get_data_from_HKMA(fps_val))

#Convert dataframe to dataframe long format
fps_val_dfl = pd.melt(fps_val_df, id_vars = ['end_of_month'], value_vars =['rtctp_pp_payee_proxy_id', 'rtctp_pp_payee_acc_no',
       'rtctp_other_payment', 'rtctp', 'rtddp', 'rt_payment', 'bat_payment',
       'total'],var_name='var', value_name='Value in HK$ thousand' )

fig = px.area(fps_val_dfl, x='end_of_month' , y='Value in HK$ thousand', color = 'var')

fig.update_layout(title_text="Turnover of HKD FPS payment (Value)(3.12.4)")

fig.show()


## Aggregate Balance

The sum of balances in the clearing accounts and reserve accounts maintained by commercial banks with the central bank. In Hong Kong, this refers to the sum of the balances in the clearing accounts maintained by the banks with the HKMA for settling interbank payments and payments between banks and the HKMA. The Aggregate Balance is a part of the Monetary Base.

In [ ]:
fig = px.line(mbc_df, x="end_of_month", y = "aggr_balance_bf_disc_win",
              title='AB: Aggregate Balance before Discount Window')

fig.show()

## Hong Kong Interbank Offered Rate (HIBOR)

The rate of interest offered on Hong Kong dollar loans by banks in the interbank market for a specified period ranging from overnight to one year.


In [ ]:
HIBOR = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/hk-interbank-ir-endperiod?sortby=end_of_month&sortorder=asc'
HIBOR_df = pd.json_normalize(get_data_from_HKMA(HIBOR))

fig = px.line(HIBOR_df, x="end_of_month", y = "ir_1m",
              title='Hong Kong Interbank Offered Rate (HIBOR)')
fig.show()
